# Laboratory work #2

Import all significant libraries for this project.

In [ ]:
# Import scikit-learn libraries


# Import other libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#### Downloading Dataset
Our dataset is Possible Asteroid Impacts with Earth (from [kaggle](https://www.kaggle.com/datasets/nasa/asteroid-impacts))

## Preprocessing Dataset